In [1]:
import sys
import os

# 현재 작업 디렉토리를 가져옵니다
current_dir = os.getcwd()

# 원하는 디렉토리의 절대 경로를 구성합니다
target_path = os.path.abspath(os.path.join(current_dir, '..', 'adas', 'pytorch-nested-unet'))

# sys.path에 추가합니다
sys.path.append(target_path)


In [ ]:
from PIL import Image
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import torch
import yaml
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator
import archs

%matplotlib inline


In [ ]:
# 필요한 경로 설정
pjt_folder = os.path.abspath(os.getcwd())
od_model = os.path.join(pjt_folder, 'models', 'adas_car_detection.pt')
ss_model = os.path.join(pjt_folder, 'models', 'adas_lane_segmentation.pth')
yml_path = os.path.join(pjt_folder, 'models', 'config.yml')
image_folder = os.path.join(pjt_folder, 'dataset', 'images')
output_image_folder = os.path.join(pjt_folder, 'dataset', 'processed_images')

# 색상 정의
color_dict = {'car': (255, 0, 0), 'pedestrian': (0, 0, 255), 'motorcycle': (255, 0, 0)}

# YAML 파일 로드
with open(yml_path) as f:
    data = yaml.load(f, Loader=yaml.FullLoader)
print(data)

print(pjt_folder)

In [ ]:
# 세그멘테이션 모델 로드
DEVICE = torch.device("cuda")
model = archs.__dict__[data['arch']](data['num_classes'], data['input_channels'], data['deep_supervision'])
model = model.to(DEVICE)
model.load_state_dict(torch.load(ss_model, map_location=DEVICE))
print("Segmentation model loaded")

# 객체 탐지 모델 로드
yolo_model = YOLO(od_model)
yolo_model.to(DEVICE)
print("YOLO Model Loaded")

In [5]:
# 차선 그리기 함수
def draw_lane(ori_img, model):
    img = cv2.resize(ori_img, (512, 256))
    input = img.astype('float32') / 255
    input = np.expand_dims(input, axis=0)
    input = torch.from_numpy(input).to(DEVICE)
    input = input.permute(0, 3, 1, 2)
    output = model(input)
    output = torch.sigmoid(output)
    output = output.permute(0, 2, 3, 1).cpu().detach()

    pred = np.array(output[0]) * 255
    pred_final = pred[:, :, 0] + pred[:, :, 1]
    pred_final = cv2.resize(pred_final, (ori_img.shape[1], ori_img.shape[0]))
    _, pred_final = cv2.threshold(pred_final, 250, 255, cv2.THRESH_BINARY)

    lane = np.zeros_like(ori_img)
    lane[:, :, 1] = pred_final
    result = cv2.addWeighted(lane, 0.9, ori_img, 1.0, 0.0)
    return result

In [6]:
# 중앙 구역 정의 (차선 사이의 영역)
def is_within_center_zone(box, frame_width, frame_height):
    x_min, y_min, x_max, y_max = box
    center_x = frame_width // 2
    center_y = frame_height // 2
    width = x_max - x_min
    height = y_max - y_min
    
    # 중앙 영역의 좌표
    central_zone_x_min = center_x - width // 2
    central_zone_x_max = center_x + width // 2
    central_zone_y_min = center_y - height // 2
    central_zone_y_max = center_y + height // 2
    
    # 박스가 중앙 영역에 포함되는지 확인
    return not (x_max < central_zone_x_min or x_min > central_zone_x_max or y_max < central_zone_y_min or y_min > central_zone_y_max)

In [ ]:
# 이미지 파일 처리
for image_file in os.listdir(image_folder):
    image_path = os.path.join(image_folder, image_file)
    ori_img = cv2.imread(image_path)
    if ori_img is None:
        continue
    
    # 차선 그리기
    result_lane = draw_lane(ori_img, model)

    # 객체 탐지
    results = yolo_model(ori_img)

    black_img = np.zeros_like(result_lane)
    warning_status = False

    for result in results:
        annotator = Annotator(ori_img)
        boxes = result.boxes
        for box in boxes:
            b = box.xyxy[0]
            cls = int(box.cls)
            p_start, p_end = (int(b[0]), int(b[1])), (int(b[2]), int(b[3]))

            if is_within_center_zone(b, ori_img.shape[1], ori_img.shape[0]):
                cv2.rectangle(black_img, p_start, p_end, color=color_dict[yolo_model.names[cls]], thickness=-1)
                
                # 경고 조건: 객체가 하단 5% 이상 위치하거나, 감지된 객체가 오토바이인 경우
                if int(b[3]) > black_img.shape[0] * 0.95 or yolo_model.names[cls] == 'motorcycle':
                    warning_status = True

    # 결과 이미지 생성
    final_result = cv2.addWeighted(black_img, 0.6, result_lane, 1.0, 0.0)
    
    if warning_status:
        cv2.rectangle(final_result, (0, 0), (600, 140), (255, 0, 0), -1, cv2.LINE_AA)
        cv2.putText(final_result, 'Warning!', (0, 100), cv2.FONT_HERSHEY_DUPLEX, 4, (255, 255, 255), thickness=3, lineType=cv2.LINE_AA)

    # 결과 이미지 저장
    output_image_path = os.path.join(output_image_folder, f"processed_{image_file}")
    cv2.imwrite(output_image_path, final_result)

print("Image processing completed.")